# <center>  Основы NLP </center>
<center><img src="https://miro.medium.com/max/2672/1*_Nb5AADlqVQJDa0YyNFKGA.jpeg"></center>
<p style="text-align: right;"><em>Aвтор:</em> Ян Шинкевич</p>

## <center> Часть 4. Машинное обучение. Сентимент-анализ

<font color='black'> На данном занятии мы продолжим тему машинного обучения и попытаемся решить проблему сентимент-анализа (анализа тональности) алгоритмами как **классического машинного обучения**, так и **глубокого** (с использованием нейросетей).
<br><br>В конце урока ты научишься строить:<br>
<b>1. Модель сентимент-анализа с помощью Logistic Regression</b><br>
<b>2. Модель сентимент-анализа с помощью LSTM нейросетей</b>

В первую очередь вспомним наш <em>универсальный</em> рецепт для любой задачи Машинного Обучения: <br>
<b>1. Датасет</b><br>
<b>2. Предобработка данных</b><br>
<b>3. Построение модели</b><br>
<b>4. Обучение модели</b><br>
<b>5. Оценка модели</b><br><br>

Поехали!

## <center>**Logistic Regression**</center>

Если в прошлом занятии мы классифицировали смс-сообщения на предмет спама, то в данном эксперименте фокус нашего внимания падёт на твиты, которые распределены по 3-ём лэйблам: <b>positive</b>, <b>negative</b>, <b>neutral</b>.

### <center>1. Датасет</center>

В первую очередь, загрузим наш датасет при помощи уже знакомой вам библиотеки <b>pandas</b>.

In [ ]:
import pandas

dataset = pandas.read_csv('polarized_twits.tsv.res', sep='\t', header=None)

Просмотрим содержимое первых 5 сэмплов:

In [ ]:
dataset.head(5)

Видим, что датасет немного грязноват и избыточен. Нас интересуют только колонки под номером 2 и 3.<br>
Давайте избавимся от колонок 0 и 1 методом <b>drop</b> и для удобства переименуем колонки <em>2</em> и <em>3</em> в <em>label</em> и <em>text</em> соответственно:

In [ ]:
dataset.drop(columns=[0, 1], axis=1, inplace = True)
dataset.rename({2:'label', 3:'twit'}, axis=1, inplace=True)

Просмотрите, как выглядит датасет самостоятельно (с помощью метода <em>head</em>):

Cделаем небольшой *EDA* (Exploratory Data Analysis) и посмотрим распределие лэйблов с помощью метода <b>value_counts</b>: 

In [ ]:
print(dataset.label.value_counts())

In [ ]:
# визуализируем
dataset.label.value_counts().plot.bar();

### <center>2. Предобработка данных</center>

В первую очередь, нам нужно нормализовать наши текстовые данные.

**Создадим функцию preprocess_data, с помощью которой посланное на вход предложение очищается от таких вещей, как стоп-слова, знаки препинания, цифры и т.д. Также функция будет производить стемминг над каждым словом:**

In [ ]:
twit = 'Theo Walcott is still shit :( watch Rafa and Johnny deal with him on Saturday...'

In [ ]:
from nltk.corpus import stopwords
from nltk import SnowballStemmer
import re
stemmer = SnowballStemmer("english")

def preprocess_twit(twit):
    
    twit = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', twit) #remove URLS
    twit = re.sub('@[^\s]+','', twit)  #remove Users
    twit = re.sub('[:;^8=]{1}-?[)PD8o]+', 'positive_smile', twit) #smiles processing
    twit = re.sub('[:;^8=]{1}-?[(/|]+', 'negative_smile', twit)
    twit = twit.strip() # удаление пробелов по бокам 
    twit = twit.lower() # lower-case
    twit = re.sub('[0-9]+', '', twit) # удаление цифр
    twit = re.sub(r'\b\w\b', '', twit) # удаление одноcимвольных токенов
    twit = re.sub('[^A-Za-zА-Яа-я_\s]+', '', twit) #удаление пунктуации
    twit = [x for x in twit.split() if x not in stopwords.words('english')] # удаление стоп-слов
    #twit = [stemmer.stem(x) for x in twit] # cтемминг

    twit = ' '.join(twit) # cоединяем элементы списка
    return twit

Посмотрим, как эта функция сработает на нашем тестовом twit:

In [ ]:
print(preprocess_twit(twit))

Чтобы применить функцию к текстовому столбику нашего датасета (twit), воспользуемся методом pandas под названием **apply** и создадим колонку с очищенными твитами (refined_twit): 

In [ ]:
dataset['refined_twit'] = dataset['twit'].apply(preprocess_twit)

Посмотрим, что у нас получилось. Выведите первые 10 сэмплов (через метод <em>head</em>):

**Разбиение датасета**

В первую очередь, отделим иксы от игреков. Затем разобьём все данные на <b>train</b> и <b>test</b> выборки в соотношении 0.7/0.3.<br>

In [ ]:
from sklearn.model_selection import train_test_split

# отделяем лейблы от фич 
X = dataset.refined_twit
y = dataset.label

# разбиваем наш датасет на train и test 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3)

**Векторизация данных**

Если в прошлом занятии мы обращались только к <b>Bag of Words</b> векторам, то в этом давайте попробуем использовать и <b>Word Embeddings</b>.<br><br>
##### Bag of Words

Чтобы не тратить много времени, Bag of Words векторизацию осуществим только с помощью бинарных векторов.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # импортируем векторизатор

vectorizer_binary = CountVectorizer(binary=True) # указываем параметр "binary" = True (отсутствие/присутствие слова)
vectorizer_binary.fit(X) # скармливаем нашему векторизатору все (!) данные
X_binary_train = vectorizer_binary.transform(X_train) # векторизируем train partition 
X_binary_test = vectorizer_binary.transform(X_test) # векторизируем test partition

Ради интереса посмотрим, какой <b>размерности</b> наши вектора и <b>какие фичи</b> использует наш векторизатор:

In [ ]:
print("Матрица train выборки:", X_binary_train.get_shape())
print("Первый вектор:\n", X_binary_train[0])

In [ ]:
print("Наши фичи:", vectorizer_binary.get_feature_names())

##### Word Embeddings

Как мы уже знаем, за Word embeddings отвечает библиотека <b>gensim</b>, делающая процесс манипуляции над ними быстрым и удобным.

In [ ]:
# импортируем библиотеку 
import gensim.downloader as api

# загружаем вектора GloVe
glove_model = api.load('glove-twitter-50')

In [ ]:
import numpy as np 

def twit_vector(doc):
    # Создаём вектор всего твита усреднением векторов всех слов в твите. Удаляем слово, если его нет в glove_model"""
    doc = [word for word in doc.split() if word in glove_model.vocab]
    return np.mean(glove_model[doc], axis=0)

In [ ]:
twit_vector(X[0])

Обернём в эмбеддинги все <b>train</b> и <b>test</b> твиты:

In [ ]:
X_train_we = X_train.apply(twit_vector)
X_test_we = X_test.apply(twit_vector)

### <center>3. Построение, обучение и оценка модели</center>

Как и планировали, используем <b>Логистическую регрессию</b> как пример алгоритма <em>классического машинного обучения</em>. <br>Cначала с <b>BoW векторами</b>:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_model = LogisticRegression(solver = 'lbfgs', multi_class='auto')
log_model.fit(X_binary_train, y_train)
predictions = log_model.predict(X_binary_test)

print("Logstic Regression with BoW accuracy score:", accuracy_score(y_test, predictions))

А теперь накормим Логистическую регрессию нашими <b>Word embeddings</b>:

In [ ]:
log_model = LogisticRegression(solver = 'lbfgs', multi_class='auto')
log_model.fit(list(X_train_we), y_train)
predictions = log_model.predict(list(X_test_we))

print("Logstic Regression with Word Embeddings accuracy score:", accuracy_score(y_test, predictions))

## <br><br><center> LSTM Neural Networks

С классическим машинным обучением мы справились и увидели, на что оно способно. <br>
Думаю, теперь нам стоить посмотреть одним глазком, какие результаты нам могут выдать нейросетевые архитектуры. А именно - <b>LSTM</b> (Long Short Term Memory), которая умеет запоминать <b>последовательности</b> токенов, а не только их наличие.

В нейросетях подход к обработке текста немного иной, в отличие от классического МО.<br>
Поэтому здесь применим иную логику.<br>
Для работы с нейросетями и их производными существует библиотека <b>keras</b>, с помощью которой творить нейросети легко и удобно

In [ ]:
# импортируем Tokenizer (грубо говоря, аналог векторизатора)
from keras.preprocessing.text import Tokenizer
# pad_sequences позволит уравнять все предложения по одной длине
from keras.preprocessing.sequence import pad_sequences
# библиотека для работы с математическими объектами
import numpy

Создадим объект <em>Tokenizer</em> и поместим его в переменную <em>t</em>. Методом <b>fit_on_texts</b> поместим туда все наши твиты, находящиеся в переменной X.

In [ ]:
t = Tokenizer()
t.fit_on_texts(X)
# Посмотрим с помощью метода word_index, что представляет собой "накормленный" Tokenizer 
print(t.word_index)

Узнаем размер нашего словаря (количество уникальных слов), а потом преобразуем две выборки в численный формат:

In [ ]:
# Размер нашего словаря (пригодится потом)
vocab_size = len(t.word_index) + 1

# Преобразование наших выборок в численный формат (где число = соответствующее ему слово)
X_train_encoded = t.texts_to_sequences(X_train)
X_test_encoded = t.texts_to_sequences(X_test)

Просмотрим, как теперь представлены наши документы (твиты):

In [ ]:
print(*X_train_encoded[:5], sep='\n')

Уравниваем все документы по самому длинному документу (переменная <b>max_length</b>):

In [ ]:
# Cамый длинный документ
max_length = max([len(twit) for twit in X])

# Пэдим каждую выборку
X_train_padded = pad_sequences(X_train_encoded, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_encoded, maxlen=max_length, padding='post')

Тут мы создадим <b>матрицу эмбдеддингов</b>, которая будет нужна для того, чтобы при поступлении в нейронную сеть наши токены оборачивались в соответствующий им эмбеддинг, а потом в этом же виде проходили дальше по слоям нейросети.

In [ ]:
# матрица эмбеддингов размера vocab_size X 50 (размер наших glove векторов), заполненная нулями
embedding_matrix = numpy.zeros((vocab_size, 50))

# заполнение матрицы реальными векторами
for word, i in t.word_index.items():
    embedding_vector = glove_model[word] if word in glove_model.vocab else None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Кроме того, переведём наши лэйблы в дискретные величины (этого требует нейросеть):

In [ ]:
y_train = y_train.replace({'positive':0,'neutral':1,'negative':2})
y_test = y_test.replace({'positive':0,'neutral':1,'negative':2})

<br><br>Построение нейросетевой модели

In [ ]:
from keras.layers import Dense, Embedding,LSTM
from keras.models import Sequential

model = Sequential()

# Input / Embdedding
model.add(Embedding(vocab_size, output_dim = 50, 
                    weights=[embedding_matrix], 
                    mask_zero=True, 
                    trainable=True))

model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=False))

# Output layer
model.add(Dense(3, activation='sigmoid'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Обучение нейросети на 2 эпохах и получение конечного <b>accuracy</b>:

In [ ]:
# кладём X и y в нашу нейросеть
model.fit(X_train_padded, y_train, epochs=2, batch_size=64)

# смотрим качество предсказаний, сделанных нейросетью на тестовой выборке
loss, accuracy = model.evaluate(X_test_padded, y_test, verbose=0)

# наша accuracy
print('\nAccuracy: {}'.format(accuracy*100))

<br><br><br>
<b>Вот такие, и огромное множество других задач обработки естественного языка (и не только) решает Machine Learning. <br>Сегодня ей под силу действительно целая масса реальных проблем разного типа, содержания и важности. Можно сказать, что современный мир постепенно "подсаживается" на данную область, о чём, вы, постоянно находясь в интернет-пространстве, наверное, и сами знаете :)<br><br>
<center>Круто, что не поленились пройти этот курс вместе с его авторами! Эти самые авторы искренне хотят, чтобы в нашем университете понимание Обработки Ествественного Языка сдвинулось с давно уже устарешвего уровня на более-менее современный и интересный!<br> Однако не стоит забывать, что мы попытались дать сильный и первоначальный толчок, открыв только ОСНОВЫ(!), с расчетом на то, что углубление подобными вещами происходить самостоятельно, на началах вашего собственного интереса :) </center><br><br>
<center>Спасибо за внимание!</center>